<a href="https://colab.research.google.com/github/Algostu/neural/blob/master/Basic_NLP_Modeling_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data 불러오기

In [0]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 7.51 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [0]:
import pandas as pd
import numpy as np
train_data = pd.read_table('/content/nsmc/ratings_train.txt')
test_data = pd.read_table('/content/nsmc/ratings_test.txt')

In [0]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


## Data 정제하기

In [0]:
# 중복 제거
train_data.drop_duplicates(subset=['document'], inplace=True)
train_data['label'].value_counts().plot(kind = 'bar')

In [0]:
print(train_data.isnull().values.any())
print(train_data.isnull().sum())

train_data.loc[train_data.document.isnull()]

In [0]:
# Null값 존재 행 제거
train_data = train_data.dropna(how='any')

In [0]:
# 정규화 한글, 영어, 띄어쓰기만 남김
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")

In [0]:
# 정규화로 null값이 다시 생겼을 수 있으니 Null값 제거 다시 진행
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how='any')

In [0]:
print('len of train data :', len(train_data))

len of train data : 146009


In [0]:
# 테스트 데이터도 똑같이 진행
test_data.drop_duplicates(subset = ['document'], inplace=True)
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z ]","")
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how='any')
print('len of test data :',len(test_data))

len of test data : 49079


## Tokenize

In [0]:
from konlpy.tag import Mecab
from tqdm.notebook import tqdm
mecab = Mecab()

In [0]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
X_train = []
for text in tqmd(train_data['document']):
  temp = []
  temp = mecab.morphs(text)
  temp = [word for word in temp if not word in stopwords]
  X_train.append(temp)

In [0]:
X_test = []
for text in tqdm(test_data['document']):
  temp = []
  temp = mecab.morphs(text)
  temp = [word for word in temp if not word in stopwords]
  X_test.append(temp)

In [0]:
# 정수 인코딩
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

In [0]:
# 출연단어의 비중 확인하기
threshold = 3
total_cnt = len(tokenizer.word_index)
rare_cnt = 0
total_freq = 0
rare_freq = 0

52169

https://wikidocs.net/44249